# Final Project - Zarina

## Choosing the models
I wanted to focus on kmer data. Both of the models I chose are trained on kmers instead of presence/absence. I was originally interested in the gradient boosting model, and due to the processing power availability to me, chose to use basic decision trees as my base estimator. I did not want to use the linear models as I felt like I could not guarantee that the kmers would fit the linear regression assumptions. For curiosities sake, I also chose to do a random forest model to see how it compares to the gradient boosting model with the same base estimator.

I am using KNN (K Nearest Neighbour) as a baseline model. In week 4, we found that it has a peak accuracy of 84%, therefore, I am hoping to generate a model with an accuracy greater than 84%.

### Imports

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import tree
from sklearn import ensemble
import bayes_opt

### Load data

In [2]:
seed = 130

def load_data():
    
    # Load Kmer data
    train_kmers = np.load('../data/train_test_data/train_kmers.npy', allow_pickle=True)
    test_kmers = np.load('../data/train_test_data/test_kmers.npy', allow_pickle=True)

    # Load target data & IDs
    y_train = np.load('../data/train_test_data/y_train.npy', allow_pickle=True)
    y_train_ids = np.load('../data/train_test_data/train_ids.npy', allow_pickle=True).astype(str)
    y_test_ids = np.load('../data/train_test_data/test_ids.npy', allow_pickle=True).astype(str)
    
    return train_kmers, test_kmers, y_train, y_train_ids, y_test_ids

X_train_kmers, X_test_kmers, y_train, y_train_ids, y_test_ids = load_data()
y_train = y_train.reshape(-1)

### K-Fold CV
Below is a stratified K-Fold Cross-Validation. I chose to stratify the distribution to get the best mix possible for my training data.

In [3]:
K = 3

kfold = sklearn.model_selection.StratifiedKFold(
    n_splits = K,
    shuffle = True, # Want to shuffle as seen in slides
    random_state = seed, # To ensure reproducible results
)

kfold_dfs = {}
val_idx = {}
for i, (train_index, val_index) in enumerate(kfold.split(X_train_kmers, y_train)):
    
    val_idx[i] = val_index

    kfold_dfs[i] = (X_train_kmers[train_index], X_train_kmers[val_index], y_train[train_index], y_train[val_index])

# Specify fold 0
X_train_fold_0 = kfold_dfs[0][0]
X_val_fold_0 = kfold_dfs[0][1]
y_train_fold_0 = kfold_dfs[0][2]
y_val_fold_0 = kfold_dfs[0][3]

# Specify fold 1
X_train_fold_1 = kfold_dfs[1][0]
X_val_fold_1 = kfold_dfs[1][1]
y_train_fold_1 = kfold_dfs[1][2]
y_val_fold_1 = kfold_dfs[1][3]

# Specify fold 2
X_train_fold_2 = kfold_dfs[2][0]
X_val_fold_2 = kfold_dfs[2][1]
y_train_fold_2 = kfold_dfs[2][2]
y_val_fold_2 = kfold_dfs[2][3]

## Building a Random Forest Model
A random forest is more robust than a simple decision tree. The hyperparameters of interest are the number of trees in the forest (n_estimators) and the tree depth (max_depth) .

In [4]:
def random_forest_model_fit_0(numberTrees, sizeTrees):
     # Create model
    randomForestModel = ensemble.RandomForestClassifier(n_estimators=int(numberTrees), max_depth=int(sizeTrees), random_state = seed)
    
    # Fit model
    randomForestModel.fit(X_train_fold_0, y_train_fold_0)

    # Evaluate the model and return the evaluation score
    score = sklearn.metrics.balanced_accuracy_score(y_val_fold_0, randomForestModel.predict(X_val_fold_0))
    
    return score

### Model Optimization
I went with a Bayesian optimizer to find the hyperparameters for my random forest model.

In [5]:
# Bounded region of parameter space
parameter_limits = {'numberTrees': (1, 50), 'sizeTrees': (1, 25)}

optimizer0 = bayes_opt.BayesianOptimization(
    f = random_forest_model_fit_0,
    pbounds = parameter_limits,
    random_state=seed,
)

# Fit the model using our custom optimizer
optimizer0.maximize(
    init_points=10, # Arbitrary larger number to increase spread
    n_iter=20, # Arbitrary large-ish number to optimize search
)

|   iter    |  target   | number... | sizeTrees |
-------------------------------------------------
| 1         | 0.8125    | 7.94      | 8.477     |
| 2         | 0.8325    | 28.18     | 4.019     |
| 3         | 0.8325    | 25.2      | 23.11     |
| 4         | 0.8425    | 19.61     | 4.035     |
| 5         | 0.835     | 49.94     | 19.5      |
| 6         | 0.82      | 36.78     | 1.759     |
| 7         | 0.82      | 40.71     | 2.236     |
| 8         | 0.855     | 28.99     | 9.147     |
| 9         | 0.815     | 11.67     | 19.34     |
| 10        | 0.8375    | 47.44     | 17.95     |
| 11        | 0.815     | 25.69     | 12.11     |
| 12        | 0.82      | 29.31     | 8.021     |
| 13        | 0.855     | 29.0      | 9.161     |
| 14        | 0.855     | 28.08     | 9.161     |
| 15        | 0.855     | 28.19     | 9.92      |
| 16        | 0.825     | 29.33     | 10.86     |
| 17        | 0.855     | 27.1      | 9.762     |
| 18        | 0.815     | 26.64     | 8.746     |


### Best Random Forest Model for Fold 0
Based on the above, the best model on fold 0 has a forest with a size of 28, and a tree depth of 9, resulting in an accuracy of 0.855. 

I ran the model on the full set to submit to Kaggle, with a private/public score of 0.95625/0.91875.

In [6]:
randomForestFold0 = ensemble.RandomForestClassifier(n_estimators=28, max_depth=9, random_state = seed)
randomForestFold0.fit(X_train_kmers, y_train)

# Make test predictions and save out as a dataframe
test_preds = randomForestFold0.predict(X_test_kmers)

# Save
test_preds_df = pd.DataFrame(data={"genome_id":y_test_ids, "y_pred":test_preds})
test_preds_df.to_csv("randomForestFold0.csv", index=False) # IMPORTANT: Do not save the index
test_preds_df.head()

,genome_id,y_pred
0,562.42833,R
1,562.42739,R
2,562.22823,S
3,562.45646,S
4,562.22547,S


### Modeling the Random Forest Model on Fold 1

In [7]:
def random_forest_model_fit_1(numberTrees, sizeTrees):
    
    randomForestModel = ensemble.RandomForestClassifier(n_estimators=int(numberTrees), max_depth=int(sizeTrees), random_state = seed)
    randomForestModel.fit(X_train_fold_1, y_train_fold_1)
    score = sklearn.metrics.balanced_accuracy_score(y_val_fold_1, randomForestModel.predict(X_val_fold_1))

    return score

# We do not need to restate the parameter limits as they are the same

optimizer1 = bayes_opt.BayesianOptimization(
    f = random_forest_model_fit_1,
    pbounds = parameter_limits,
    random_state=seed,
)

optimizer1.maximize(
    init_points=10, # Arbitrary larger number to increase spread
    n_iter=20, # Arbitrary large-ish number to optimize search
)

|   iter    |  target   | number... | sizeTrees |
-------------------------------------------------
| 1         | 0.8473    | 7.94      | 8.477     |
| 2         | 0.8797    | 28.18     | 4.019     |
| 3         | 0.8498    | 25.2      | 23.11     |
| 4         | 0.8497    | 19.61     | 4.035     |
| 5         | 0.8723    | 49.94     | 19.5      |
| 6         | 0.8473    | 36.78     | 1.759     |
| 7         | 0.8273    | 40.71     | 2.236     |
| 8         | 0.8648    | 28.99     | 9.147     |
| 9         | 0.8948    | 11.67     | 19.34     |
| 10        | 0.8623    | 47.44     | 17.95     |
| 11        | 0.8897    | 10.22     | 22.55     |
| 12        | 0.8823    | 6.415     | 18.96     |
| 13        | 0.8748    | 15.56     | 22.17     |
| 14        | 0.8049    | 1.188     | 24.89     |
| 15        | 0.8773    | 15.44     | 14.95     |
| 16        | 0.8798    | 40.44     | 24.95     |
| 17        | 0.8623    | 47.79     | 24.85     |
| 18        | 0.8798    | 36.14     | 19.88     |


### Best Random Forest Model for Fold 1
This suggests that the best model for fold 1 would be a forest with a size of 11, and a tree depth of 19, resulting in an accuracy of 0.8948

Again I saved the results here to submit to Kaggle, which predicted a score of 0.95000/0.90000.

In [8]:
randomForestFold1 = ensemble.RandomForestClassifier(n_estimators=11, max_depth=19, random_state = seed)
randomForestFold1.fit(X_train_kmers, y_train)

test_preds = randomForestFold1.predict(X_test_kmers)

test_preds_df = pd.DataFrame(data={"genome_id":y_test_ids, "y_pred":test_preds})
test_preds_df.to_csv("randomForestFold1.csv", index=False) # IMPORTANT: Do not save the index
test_preds_df.head()

,genome_id,y_pred
0,562.42833,R
1,562.42739,R
2,562.22823,S
3,562.45646,S
4,562.22547,S


### Modeling the Random Forest Model on Fold 2

In [9]:
def random_forest_model_fit_2(numberTrees, sizeTrees):
    randomForestModel = ensemble.RandomForestClassifier(n_estimators=int(numberTrees), max_depth=int(sizeTrees), random_state = seed)
    randomForestModel.fit(X_train_fold_2, y_train_fold_2)
    score = sklearn.metrics.balanced_accuracy_score(y_val_fold_2, randomForestModel.predict(X_val_fold_2))

    return score

optimizer2 = bayes_opt.BayesianOptimization(
    f = random_forest_model_fit_2,
    pbounds = parameter_limits,
    random_state=seed,
)

optimizer2.maximize(
    init_points=10, # Arbitrary larger number to increase spread
    n_iter=20, # Arbitrary large-ish number to optimize search
)

|   iter    |  target   | number... | sizeTrees |
-------------------------------------------------
| 1         | 0.7699    | 7.94      | 8.477     |
| 2         | 0.7899    | 28.18     | 4.019     |
| 3         | 0.7524    | 25.2      | 23.11     |
| 4         | 0.7799    | 19.61     | 4.035     |
| 5         | 0.7649    | 49.94     | 19.5      |
| 6         | 0.7773    | 36.78     | 1.759     |
| 7         | 0.7749    | 40.71     | 2.236     |
| 8         | 0.7849    | 28.99     | 9.147     |
| 9         | 0.7524    | 11.67     | 19.34     |
| 10        | 0.7649    | 47.44     | 17.95     |
| 11        | 0.6698    | 1.0       | 1.0       |
| 12        | 0.7974    | 49.98     | 5.215     |
| 13        | 0.7298    | 1.068     | 24.92     |
| 14        | 0.7773    | 49.87     | 1.143     |
| 15        | 0.7749    | 38.73     | 10.92     |
| 16        | 0.7749    | 50.0      | 10.65     |
| 17        | 0.7649    | 38.8      | 25.0      |
| 18        | 0.7624    | 18.55     | 11.59     |


### Best Random Forest Model for Fold 2
This suggests that the best model for fold 2 would be a forest with a size of 36, and a tree depth of 6, resulting in an accuracy of 0.8149

Kaggle had a score of 0.91875/0.87500

In [10]:
randomForestFold2 = ensemble.RandomForestClassifier(n_estimators=36, max_depth=6, random_state = seed)
randomForestFold2.fit(X_train_kmers, y_train)

test_preds = randomForestFold2.predict(X_test_kmers)

test_preds_df = pd.DataFrame(data={"genome_id":y_test_ids, "y_pred":test_preds})
test_preds_df.to_csv("randomForestFold2.csv", index=False) # IMPORTANT: Do not save the index
test_preds_df.head()

,genome_id,y_pred
0,562.42833,R
1,562.42739,R
2,562.22823,S
3,562.45646,S
4,562.22547,S


### Final Random Forest Model
Based on the results of the three optimized models above, the best model would be the one trained on fold 1. (Forest size of 11, and a tree depth of 19) as it had the highest accuracy of 0.8948

## Building a Gradient Boost Model

In [11]:
def gradient_boost_model_fit_0(numberTrees, sizeTrees, learnRate):
    # Create model
    gradientBoostModel = sklearn.ensemble.AdaBoostClassifier(
        estimator = tree.DecisionTreeClassifier(max_depth=int(sizeTrees)), # Can choose any simple base estimator
        n_estimators = int(numberTrees),
        learning_rate = float(learnRate), # Another parameter to tune
        algorithm="SAMME",
    )
    
    # Fit model
    gradientBoostModel.fit(X_train_fold_0, y_train_fold_0)

    # Evaluate the model and return the evaluation score
    score = sklearn.metrics.balanced_accuracy_score(y_val_fold_0, gradientBoostModel.predict(X_val_fold_0))
    
    return score

### Gradient Boost Model Optimization
Through trial and error, I found a rough upper bound of the learning rate of approximately 25

In [12]:
# Bounded region of parameter space
parameter_limits = {'numberTrees': (1, 50), 'sizeTrees': (1, 25), 'learnRate': (1.0, 25.0)}

optimizer0 = bayes_opt.BayesianOptimization(
    f = gradient_boost_model_fit_0,
    pbounds = parameter_limits,
    random_state=seed,
)

# Fit the model using our custom optimizer
optimizer0.maximize(
    init_points=10, # Arbitrary larger number to increase spread
    n_iter=20, # Arbitrary large-ish number to optimize search
)

|   iter    |  target   | learnRate | number... | sizeTrees |
-------------------------------------------------------------
| 1         | 0.7875    | 4.399     | 16.27     | 14.32     |
| 2         | 0.7925    | 4.019     | 25.2      | 23.11     |
| 3         | 0.795     | 10.12     | 7.196     | 24.97     |
| 4         | 0.82      | 19.5      | 36.78     | 1.759     |
| 5         | 0.795     | 20.45     | 3.523     | 14.71     |
| 6         | 0.785     | 9.147     | 11.67     | 19.34     |
| 7         | 0.8175    | 23.75     | 35.6      | 23.36     |
| 8         | 0.78      | 11.53     | 31.72     | 17.02     |
| 9         | 0.7925    | 2.883     | 4.284     | 12.6      |
| 10        | 0.795     | 22.25     | 40.26     | 18.79     |
| 11        | 0.82      | 21.83     | 38.81     | 1.0       |


/home/zarina/miniconda3/envs/etbg-env/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:695: RuntimeWarning: overflow encountered in exp
  sample_weight = np.exp(
/home/zarina/miniconda3/envs/etbg-env/lib/python3.10/site-packages/sklearn/base.py:1474: UserWarning: Sample weights have reached infinite values, at iteration 1, causing overflow. Iterations stopped. Try lowering the learning rate.
  return fit_method(estimator, *args, **kwargs)


| 12        | 0.255     | 25.0      | 32.69     | 1.0       |
| 13        | 0.6875    | 19.77     | 40.5      | 3.821     |
| 14        | 0.8075    | 22.83     | 35.79     | 20.38     |
| 15        | 0.795     | 20.24     | 38.59     | 23.43     |
| 16        | 0.77      | 19.05     | 33.18     | 22.96     |
| 17        | 0.785     | 25.0      | 40.54     | 23.52     |
| 18        | 0.82      | 14.17     | 36.1      | 1.703     |
| 19        | 0.7025    | 12.86     | 32.6      | 5.656     |
| 20        | 0.82      | 11.64     | 40.29     | 1.045     |
| 21        | 0.82      | 6.95      | 37.9      | 1.861     |
| 22        | 0.82      | 6.795     | 43.18     | 1.0       |
| 23        | 0.8025    | 8.417     | 41.5      | 5.937     |
| 24        | 0.695     | 2.074     | 40.86     | 3.639     |
| 25        | 0.7125    | 10.44     | 46.75     | 2.107     |
| 26        | 0.8075    | 25.0      | 30.73     | 21.64     |
| 27        | 0.7875    | 25.0      | 32.09     | 16.18     |
| 28    

### Best Gradient Boost Model for Fold 0
Based on the above, the best model on fold 0 has a forest with a size of 36, a tree depth of 1, and a learning rate of 19.5 resulting in an accuracy of 0.82

Kaggle score: 0.85625/0.83125

In [13]:
gradientBoostFold0 = sklearn.ensemble.AdaBoostClassifier(estimator = tree.DecisionTreeClassifier(max_depth=1), n_estimators = 36, learning_rate = 19.5, algorithm="SAMME")

gradientBoostFold0.fit(X_train_kmers, y_train)

# Make test predictions and save out as a dataframe
test_preds = gradientBoostFold0.predict(X_test_kmers)

# Save
test_preds_df = pd.DataFrame(data={"genome_id":y_test_ids, "y_pred":test_preds})
test_preds_df.to_csv("gradientBoostFold0.csv", index=False) # IMPORTANT: Do not save the index
test_preds_df.head()

,genome_id,y_pred
0,562.42833,S
1,562.42739,R
2,562.22823,S
3,562.45646,S
4,562.22547,S


Let's repeat the same steps for the other two folds.

In [14]:
def gradient_boost_model_fit_1(numberTrees, sizeTrees, learnRate):

    gradientBoostModel = sklearn.ensemble.AdaBoostClassifier(
        estimator = tree.DecisionTreeClassifier(max_depth=int(sizeTrees)), # Can choose any simple base estimator
        n_estimators = int(numberTrees),
        learning_rate = float(learnRate), # Another parameter to tune
        algorithm="SAMME",
    )
    
    gradientBoostModel.fit(X_train_fold_1, y_train_fold_1)
    score = sklearn.metrics.balanced_accuracy_score(y_val_fold_1, gradientBoostModel.predict(X_val_fold_1))
    
    return score

optimizer1 = bayes_opt.BayesianOptimization(
    f = gradient_boost_model_fit_1,
    pbounds = parameter_limits,
    random_state=seed,
)

optimizer1.maximize(
    init_points=10, # Arbitrary larger number to increase spread
    n_iter=20, # Arbitrary large-ish number to optimize search
)

|   iter    |  target   | learnRate | number... | sizeTrees |
-------------------------------------------------------------
| 1         | 0.8022    | 4.399     | 16.27     | 14.32     |
| 2         | 0.8247    | 4.019     | 25.2      | 23.11     |
| 3         | 0.8047    | 10.12     | 7.196     | 24.97     |
| 4         | 0.8423    | 19.5      | 36.78     | 1.759     |
| 5         | 0.7997    | 20.45     | 3.523     | 14.71     |
| 6         | 0.8372    | 9.147     | 11.67     | 19.34     |
| 7         | 0.8272    | 23.75     | 35.6      | 23.36     |
| 8         | 0.8147    | 11.53     | 31.72     | 17.02     |
| 9         | 0.7847    | 2.883     | 4.284     | 12.6      |
| 10        | 0.8272    | 22.25     | 40.26     | 18.79     |
| 11        | 0.6865    | 18.85     | 36.03     | 2.146     |
| 12        | 0.8172    | 20.04     | 24.73     | 18.9      |
| 13        | 0.581     | 18.7      | 11.51     | 2.163     |
| 14        | 0.6865    | 14.73     | 46.53     | 2.709     |
| 15    

### Best Gradient Boost Model for Fold 1
Based on the above, the best model on fold 1 has a forest with a size of 25, a tree depth of 23, and a learning rate of 4.019 resulting in an accuracy of 0.8523

Kaggle score: 0.86250/0.86250

In [15]:
gradientBoostFold1 = sklearn.ensemble.AdaBoostClassifier(estimator = tree.DecisionTreeClassifier(max_depth=23), n_estimators = 25, learning_rate = 4.019, algorithm="SAMME")

gradientBoostFold1.fit(X_train_kmers, y_train)

test_preds = gradientBoostFold1.predict(X_test_kmers)

test_preds_df = pd.DataFrame(data={"genome_id":y_test_ids, "y_pred":test_preds})
test_preds_df.to_csv("gradientBoostFold1.csv", index=False) # IMPORTANT: Do not save the index
test_preds_df.head()

,genome_id,y_pred
0,562.42833,S
1,562.42739,R
2,562.22823,S
3,562.45646,S
4,562.22547,S


In [16]:
def gradient_boost_model_fit_2(numberTrees, sizeTrees, learnRate):

    gradientBoostModel = sklearn.ensemble.AdaBoostClassifier(
        estimator = tree.DecisionTreeClassifier(max_depth=int(sizeTrees)), # Can choose any simple base estimator
        n_estimators = int(numberTrees),
        learning_rate = float(learnRate), # Another parameter to tune
        algorithm="SAMME",
    )
    
    gradientBoostModel.fit(X_train_fold_2, y_train_fold_2)
    score = sklearn.metrics.balanced_accuracy_score(y_val_fold_2, gradientBoostModel.predict(X_val_fold_2))
    
    return score

optimizer2 = bayes_opt.BayesianOptimization(
    f = gradient_boost_model_fit_2,
    pbounds = parameter_limits,
    random_state=seed,
)

optimizer2.maximize(
    init_points=10, # Arbitrary larger number to increase spread
    n_iter=20, # Arbitrary large-ish number to optimize search
)

|   iter    |  target   | learnRate | number... | sizeTrees |
-------------------------------------------------------------
| 1         | 0.7274    | 4.399     | 16.27     | 14.32     |
| 2         | 0.7124    | 4.019     | 25.2      | 23.11     |
| 3         | 0.7249    | 10.12     | 7.196     | 24.97     |
| 4         | 0.7873    | 19.5      | 36.78     | 1.759     |
| 5         | 0.7149    | 20.45     | 3.523     | 14.71     |
| 6         | 0.7624    | 9.147     | 11.67     | 19.34     |
| 7         | 0.7224    | 23.75     | 35.6      | 23.36     |
| 8         | 0.7174    | 11.53     | 31.72     | 17.02     |
| 9         | 0.7249    | 2.883     | 4.284     | 12.6      |
| 10        | 0.7349    | 22.25     | 40.26     | 18.79     |
| 11        | 0.7274    | 6.716     | 23.38     | 23.29     |
| 12        | 0.7873    | 20.95     | 38.21     | 1.69      |


/home/zarina/miniconda3/envs/etbg-env/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:695: RuntimeWarning: overflow encountered in exp
  sample_weight = np.exp(
/home/zarina/miniconda3/envs/etbg-env/lib/python3.10/site-packages/sklearn/base.py:1474: UserWarning: Sample weights have reached infinite values, at iteration 2, causing overflow. Iterations stopped. Try lowering the learning rate.
  return fit_method(estimator, *args, **kwargs)


| 13        | 0.7873    | 24.32     | 33.35     | 1.0       |
| 14        | 0.7873    | 20.18     | 25.23     | 1.0       |
| 15        | 0.7873    | 15.56     | 49.8      | 1.0       |


/home/zarina/miniconda3/envs/etbg-env/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:695: RuntimeWarning: overflow encountered in exp
  sample_weight = np.exp(
/home/zarina/miniconda3/envs/etbg-env/lib/python3.10/site-packages/sklearn/base.py:1474: UserWarning: Sample weights have reached infinite values, at iteration 1, causing overflow. Iterations stopped. Try lowering the learning rate.
  return fit_method(estimator, *args, **kwargs)


| 16        | 0.2203    | 25.0      | 50.0      | 1.0       |
| 17        | 0.7873    | 8.898     | 47.68     | 1.139     |
| 18        | 0.7149    | 11.26     | 50.0      | 8.698     |
| 19        | 0.7298    | 22.45     | 29.88     | 9.005     |
| 20        | 0.7873    | 12.78     | 41.55     | 1.0       |
| 21        | 0.7873    | 10.61     | 28.2      | 1.0       |
| 22        | 0.2203    | 12.74     | 18.54     | 1.0       |
| 23        | 0.7873    | 6.695     | 34.76     | 1.629     |
| 24        | 0.7674    | 12.46     | 33.18     | 6.739     |
| 25        | 0.7349    | 1.0       | 30.76     | 8.185     |
| 26        | 0.7924    | 1.0       | 42.39     | 5.712     |
| 27        | 0.7249    | 1.0       | 41.9      | 15.26     |
| 28        | 0.7449    | 18.48     | 17.15     | 24.32     |
| 29        | 0.7274    | 1.031     | 13.35     | 23.81     |
| 30        | 0.7274    | 8.279     | 40.57     | 8.36      |


### Best Gradient Boost Model for Fold 2
Based on the above, the best model on fold 2 has a forest with a size of 36, a tree depth of 1, and a learning rate of 19.5 resulting in an accuracy of 0.7873
These are the same parameters as fold 0!

Kaggle score: 0.85625/0.83125

In [17]:
gradientBoostFold2 = sklearn.ensemble.AdaBoostClassifier(estimator = tree.DecisionTreeClassifier(max_depth=1), n_estimators = 36, learning_rate = 19.5, algorithm="SAMME")

gradientBoostFold2.fit(X_train_kmers, y_train)

test_preds = gradientBoostFold2.predict(X_test_kmers)

test_preds_df = pd.DataFrame(data={"genome_id":y_test_ids, "y_pred":test_preds})
test_preds_df.to_csv("gradientBoostFold2.csv", index=False) # IMPORTANT: Do not save the index
test_preds_df.head()



,genome_id,y_pred
0,562.42833,S
1,562.42739,R
2,562.22823,S
3,562.45646,S
4,562.22547,S


### Final Gradient Boost Model
Based on the results of the three optimized models, the best model would be the one trained on fold 1 (Forest with a size of 25, a tree depth of 23, and a learning rate of 4.019), with a resulting accuracy of 0.8523

## Conclusion
Perhaps simple is better in this case, as the random forest yielded better accuracies compared to the gradient boost across the board. The best model that I came up with was the random forest model trained on the middle fold of data.

This is, of course, only the best model in the narrow scope of the data used and the default sklearn packages.

I also did not have the chance to compare the models against each other, or to find the balanced accuracy.